In [2]:
import pandas as pd
df=pd.read_csv('/data/cluster_travel_matrix.csv')

In [6]:
import pandas as pd
import numpy as np
df=df.replace([np.inf, -np.inf], np.nan)
df=df.dropna()

In [7]:
from sklearn.cluster import KMeans

In [8]:
kmeans = KMeans(n_clusters=100).fit(df[[f for f in df.columns if f not in ['distance','stopA','stopB']]])

In [9]:
df['labels']=kmeans.labels_

In [11]:
cluster0=df[df['labels']==0]
from dbanalysis import stop_tools
to_concat = []
count=0

for row in cluster0.itertuples():
    print(count)
    fs = str(row[1])
    ts=str(row[2])
    to_concat.append(stop_tools.stop_data(fs,ts))
cluster_stops = pd.concat(to_concat,axis=0)
cluster_stops.to_csv('/data/cluser1.csv',index=False)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [63]:
df = cluster_stops

In [35]:
len(df)

228097

In [45]:
df.columns

Index(['index', 'dayofservice', 'tripid', 'plannedtime_arr_from',
       'plannedtime_dep_from', 'actualtime_arr_from', 'actualtime_dep_from',
       'plannedtime_arr_to', 'actualtime_arr_to', 'routeid', 'fromstop',
       'tostop', 'traveltime', 'dwelltime', 'distance', 'speed', 'dt', 'date',
       'day', 'month', 'hour', 'year', 'dewpt', 'msl', 'rain', 'rhum', 'temp',
       'vappr', 'wetb'],
      dtype='object')

In [64]:
to_concat = []
for hour in df['hour'].unique():
    temp = df[df['hour']==hour]
    for stopA in temp['fromstop'].unique():
        temp2 = temp[temp['fromstop']==stopA]
        for stopB in temp2['tostop'].unique():
            temp3 = temp2[temp2['tostop']==stopB]
            temp3['mean_travel_time']=temp3['traveltime'].mean()
            to_concat.append(temp3)
df=pd.concat(to_concat,axis=0)

# Classifying lateness on the cluster

In [47]:
from sklearn.ensemble import RandomForestClassifier as rf

In [65]:
df['late']=df['traveltime']>df['mean_travel_time']
df=df.dropna()

In [18]:
df.columns

Index(['index', 'dayofservice', 'tripid', 'plannedtime_arr_from',
       'plannedtime_dep_from', 'actualtime_arr_from', 'actualtime_dep_from',
       'plannedtime_arr_to', 'actualtime_arr_to', 'routeid', 'fromstop',
       'tostop', 'traveltime', 'dwelltime', 'distance', 'speed', 'dt', 'date',
       'day', 'month', 'hour', 'year', 'dewpt', 'msl', 'rain', 'rhum', 'temp',
       'vappr', 'wetb', 'mean_travel_time', 'late'],
      dtype='object')

In [66]:
train = df[df['year']==2016]
test= df[df['year']==2017]

In [67]:
clf=rf().fit(train[['actualtime_arr_from','day', 'month', 'hour', 'year', 'dewpt', 'msl', 'rain', 'rhum', 'temp',
       'vappr', 'wetb', 'mean_travel_time']+['distance']],train['late'])

In [68]:
preds = clf.predict(test[['actualtime_arr_from','day', 'month', 'hour', 'year', 'dewpt', 'msl', 'rain', 'rhum', 'temp',
       'vappr', 'wetb', 'mean_travel_time']+['distance']])

In [69]:
from sklearn.metrics import accuracy_score

In [71]:
accuracy_score(test['late'],preds)

0.5831513295656677

In [53]:
df=cluster_stops
to_concat = []
for hour in df['hour'].unique():
    temp = df[df['hour']==hour]
    temp['mean_travel_time']=temp['traveltime'].mean()
    to_concat.append(temp)
    
            
df=pd.concat(to_concat,axis=0)

/home/student/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [56]:
df['late']=df['traveltime']>df['mean_travel_time']
df=df.dropna()
train = df[df['year']==2016]
test= df[df['year']==2017]

In [72]:
clf=rf().fit(train[['hour', 'dewpt', 'msl', 'rain', 'rhum', 'temp',
       'vappr', 'wetb', 'mean_travel_time']+['distance']],train['late'])

In [73]:
preds = clf.predict(test[['hour','dewpt', 'msl', 'rain', 'rhum', 'temp',
       'vappr', 'wetb', 'mean_travel_time']+['distance']])

In [74]:
accuracy_score(test['late'],preds)

0.5714655618850336

# So we can classify for these larger means (for all stops), but not for the more specific means (just for some stops)

In [75]:
from sklearn.decomposition import PCA

In [78]:
X = PCA(n_components=10).fit_transform(df[['actualtime_arr_from','day', 'month', 'hour', 'year', 'dewpt', 'msl', 'rain', 'rhum', 'temp',
       'vappr', 'wetb', 'mean_travel_time','distance']])

In [105]:
late = np.array([[x for x in df['late']]])

In [107]:
X=pd.DataFrame(X)

In [110]:
X['late']=df['late']

ValueError: cannot reindex from a duplicate axis

In [95]:
X[0:,11:12]=[i for i in df['late']]

ValueError: could not broadcast input array from shape (228097) into shape (228097,0)

In [84]:
clf=rf().fit(X,df['late'])

In [86]:
preds=clf.predict(X)

In [87]:
accuracy_score(df['late'],preds)

0.9692016992770619